In [4]:
# 提取图像特征 
import cv2
import numpy as np
def get_points_on_line(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    dx = x2 - x1
    dy = y2 - y1
    steps = max(abs(dx), abs(dy))
    if steps == 0:
        return [(x1, y1)]
    x_inc = dx / steps
    y_inc = dy / steps
    x = x1
    y = y1
    for _ in range(steps):
        points.append((round(x), round(y)))
        x += x_inc
        y += y_inc
    points.append((x2, y2))
    return points

def extract_features(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
     # 二值化头颅图像
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # 提取头颅轮廓
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv2.contourArea)
    
    ## CI
    # 计算最大宽度和最大头部长度
    x, y, w, h = cv2.boundingRect(max_contour)
    max_width = w
    max_length = h
    # 计算颅骨指数
    CI = (max_width / max_length) * 100



    
    contour = max_contour[:, 0, :]
    rect = cv2.minAreaRect(max_contour)
    # 获取矩形的四个顶点坐标，并绘制出矩形
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    # 计算矩形的对角线交点
    p1 = (box[0][0], box[0][1])
    p2 = (box[2][0], box[2][1])
    p3 = (box[1][0], box[1][1])
    p4 = (box[3][0], box[3][1])
   #contour points array
    intersections = []
    diagonalA = get_points_on_line(p1, p2)
    diagonalB = get_points_on_line(p3, p4)
    diagonalA = np.array(diagonalA)
    diagonalB = np.array(diagonalB)
    for point1 in diagonalA :
    # 判断点是否在轮廓上
        if cv2.pointPolygonTest(contour, tuple(point1), False) == 0 :
            intersections.append(point1.astype(int))
    for  point2 in diagonalB:
            # 判断点是否在轮廓上
        if  cv2.pointPolygonTest(contour, tuple(point2),False) == 0:
            intersections.append(point2.astype(int))
    # 计算矩形的对角线交点
    d1 = intersections[0]
    d2 = intersections[1]
    d3 = intersections[2]
    d4 = intersections[3]
    # 计算对角线长度差异
    A =abs(np.linalg.norm(np.array(d2) - np.array(d1)))
    B =abs(np.linalg.norm(np.array(d3) - np.array(d4)))
    CVAI = abs(A - B) / max(A, B) * 100

    # AMWR
    p1_p4 = np.linspace(p1, p4, num=4, endpoint=False)
    p2_p3 = np.linspace(p2, p3, num=4, endpoint=False)

    # 找到第二条边的三个四等分点
    p1_p3 = np.linspace(p1, p3, num=4, endpoint=False)
    p2_p4 = np.linspace(p2, p4, num=4, endpoint=False)
    L1=np.array(get_points_on_line(p2_p3[1].astype(int), p1_p4[3].astype(int)))
    L2=np.array(get_points_on_line(p2_p3[2].astype(int), p1_p4[2].astype(int)))
    L3=np.array(get_points_on_line(p2_p3[3].astype(int), p1_p4[1].astype(int)))
     ###########
    L4=np.array(get_points_on_line(p1_p3[1].astype(int), p2_p4[1].astype(int)))
    L5=np.array(get_points_on_line(p1_p3[3].astype(int), p2_p4[3].astype(int)))
    intersections_4=[]
    for point1 in L1:
        # 判断点是否在轮廓上
        if cv2.pointPolygonTest(contour, tuple(point1), False) == 0:
            intersections_4.append(point1.astype(int))
    for point2 in L2:
        # 判断点是否在轮廓上
        if cv2.pointPolygonTest(contour, tuple(point2), False) == 0:
            intersections_4.append(point2.astype(int))
    for point3 in L3:
        # 判断点是否在轮廓上
        if cv2.pointPolygonTest(contour, tuple(point3), False) == 0:
            intersections_4.append(point3.astype(int))

    for point4 in L4:
        # 判断点是否在轮廓上
        if cv2.pointPolygonTest(contour, tuple(point4), False) == 0:
            intersections_4.append(point4.astype(int))

    for point5 in L5:
        # 判断点是否在轮廓上
        if cv2.pointPolygonTest(contour, tuple(point5), False) == 0:
            intersections_4.append(point5.astype(int))


    l1p1=intersections_4[0]
    l1p2=intersections_4[1]
    l2p1=intersections_4[2]
    l2p2=intersections_4[3]
    l3p1=intersections_4[4]
    l3p2=intersections_4[5]

    l4p1=intersections_4[6]
    l4p2=intersections_4[7]

    l5p1=intersections_4[8]
    l5p2=intersections_4[9]
    #########


    Post = abs(np.linalg.norm(np.array(l1p1) - np.array(l1p2)))
    Mid = abs(np.linalg.norm(np.array(l2p2) - np.array(l2p1)))
    Ant = abs(np.linalg.norm(np.array(l3p2) - np.array(l3p1)))
    AMWR=Ant/Mid*100
    APWR=Ant/Post*100

    L_AP = abs(np.linalg.norm(np.array(l4p2) - np.array(l5p1)))
    R_AP = abs(np.linalg.norm(np.array(l4p1) - np.array(l5p2)))
    N=min(L_AP,R_AP)
    D=max(L_AP,R_AP)
    LRHR=N/D*100

  

   # 返回特征向量
    return [CI, CVAI,AMWR,APWR,LRHR]


In [5]:
img_path="/home/ubuntu/data_process/CT_Result/N_unet/N.23.png"
extract_features(img_path)

[78.51239669421489,
 2.0200797773371346,
 88.01150135121547,
 102.97195172766085,
 98.99678184569744]

In [ ]:
# 读取数据
images = []
labels = []
for i in range(1, 11):
    for j in range(1, 6):
        image = cv2.imread('samples/s{}.{}.jpg'.format(i, j))
        images.append(image)
        labels.append(i)

# 提取特征向量
features = []
for image in images:
    feature = extract_features(image)
    features.append(feature)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

# 特征标准化
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# 训练分类器
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

# 预测测试集
y_pred = classifier.predict(X_test)

# 计算准确率
accuracy = (y_pred == y_test).sum() / len(y_test)
print('Accuracy:', accuracy)